In [1]:
from transformers import AutoTokenizer, AutoModelForPreTraining
from transformers import AutoTokenizer, AutoModelForTokenClassification
import transformers
from transformers import Trainer
from transformers import Trainer, TrainingArguments
from transformers import pipeline
from torch.utils.data import DataLoader
from datasets import load_dataset
import numpy as np
import sklearn
import evaluate
import json

In [2]:
jnlpba = load_dataset('jnlpba')
full_names_tags = {"O": "Outside", "B-DNA": "Gene Begin", "I-DNA": "Gene Inside", "B-PROTEIN": "Protein Begin", "I-PROTEIN": "Protein Inside", "B-CELL-LINE": "Cell Line Begin", "I-CELL-LINE": "Cell Line Inside", "B-CELL-TYPE": "Cell Type Begin", "I-CELL-TYPE": "Cell Type Inside", "B-DISEASE": "Disease Begin", "I-DISEASE": "Disease Inside"}

Found cached dataset jnlpba (/Users/maxhager/.cache/huggingface/datasets/jnlpba/jnlpba/1.0.0/3062f220823930cffde7976b694aa67bac3b06c322a02ced92d3761519810ce4)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained("chintagunta85/electramed-small-JNLPBA-ner")
model = AutoModelForTokenClassification.from_pretrained("chintagunta85/electramed-small-JNLPBA-ner")

nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")
nlp("Apple est créée le 1er avril 1976 dans le garage de la maison d'enfance de Steve Jobs à Los Altos en Californie par Steve Jobs, Steve Wozniak et Ronald Wayne14, puis constituée sous forme de société le 3 janvier 1977 à l'origine sous le nom d'Apple Computer, mais pour ses 30 ans et pour refléter la diversification de ses produits, le mot « computer » est retiré le 9 janvier 2015.")

# Convert the model to an instance of the AutoModelForTokenClassification class
#model = AutoModelForTokenClassification.from_pretrained(model)

#now i need to do evaluation
'''
find out and write down which scores I wanna meassure
    Loss: 0.1167
    Precision: 0.8225
    Recall: 0.8782
    F1: 0.8494
    Accuracy: 0.9621
loading validation dataset 
find out how to validate a model with validation data
save results
'''
# Load the validation data
#validation_data = jnlpba["validation"]

# Evaluate the model on the validation data
#eval_results = model.evaluate(validation_data)

# Print the evaluation results
#print(eval_results)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


'\nfind out and write down which scores I wanna meassure\n    Loss: 0.1167\n    Precision: 0.8225\n    Recall: 0.8782\n    F1: 0.8494\n    Accuracy: 0.9621\nloading validation dataset \nfind out how to validate a model with validation data\nsave results\n'

In [11]:
from evaluate import evaluator
from datasets import load_dataset
task_evaluator = evaluator("token-classification")
data = load_dataset("jnlpba", split="validation")
results = task_evaluator.compute(
    model_or_pipeline="chintagunta85/electramed-small-JNLPBA-ner",
    data=data,
    metric="seqeval",
)

Found cached dataset jnlpba (/Users/maxhager/.cache/huggingface/datasets/jnlpba/jnlpba/1.0.0/3062f220823930cffde7976b694aa67bac3b06c322a02ced92d3761519810ce4)


  0%|          | 0/7714 [00:00<?, ?ex/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/Users/maxhager/.virtualenvs/thesis/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LABEL_0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/maxhager/.virtualenvs/thesis/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LABEL_9 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/maxhager/.virtualenvs/thesis/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LABEL_10 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/maxhager/.virtualenvs/thesis/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LABEL_7 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/max

In [10]:
print(jnlpba["validation[:2]"][0])

KeyError: 'validation[:2]'

In [4]:
# model on which I want to fine tune on
# i go the dataset page and than I go to the leaderbord and see a model on the first place 
# i take this model because the leaderboard shows which model has the best scores on the dataset and i want to be state of the art
#SOTA model for NER on jnlpba is KeBioLM and achieves F1 82

#I just realize that the model is possible to evaluate like its written here https://huggingface.co/sciarrilli/biobert-base-cased-v1.2-finetuned-ner#:~:text=it%20achieves%20the%20following%20results%20on%20the%20evaluation%20set%3A
#there is now absolute no need to fine tune the model. but it could be a good introduction into doing finetuning. I basically can just copy the variables which the guy displays
#can do the fine tuning with this variables - yes lets do it!

#--> found new sota for token classification https://paperswithcode.com/sota/token-classification-on-jnlpba
#here https://huggingface.co/chintagunta85/electramed-small-JNLPBA-ner?text=My+name+is+Wolfgang+and+I+live+in+Berlin on huggingface
#was fine tuned on https://huggingface.co/giacomomiolo/electramed_small_scivocab
#the reason why i wanna fine tune is because i want to figure out how this works for my seq2seq later
